In [134]:
import joblib
import tensorflow.keras.models
import tensorflow as tf
import numpy as np
import sklearn
import pandas as pd
import datetime

In [135]:
### first model 
lin_reg=joblib.load("first_model.joblib")

### second model
second_modell= tf.keras.models.load_model("second_model")





In [136]:

def sek_to_time(sekunden):
    stunden = sekunden // 3600
    minuten = (sekunden % 3600) // 60
    restsekunden = sekunden % 60
    return stunden,minuten,restsekunden

In [137]:
dataFrameRead=pd.read_csv("schedule.csv")
dataFrameInitial=pd.DataFrame(dataFrameRead.iloc[:,:-1])


In [192]:
def f_modell(time):
    funk=(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
    time_for_modell=np.array(funk(time)).reshape(-1,1)
    output=lin_reg.predict(time_for_modell)
    output=int(output[0,0])
    return (output-time_for_modell) # the output will be the check-out of one person so substract to calc the period omne patient stays

def sec_modell(month,day):
    new_row=(month,day)
    dataFrameInitial.loc[len(dataFrameInitial)] = new_row
    dataFrameLatest=pd.get_dummies(dataFrameInitial)
    dataFrameLatestAfter=tf.convert_to_tensor(dataFrameLatest,dtype=tf.int64)
    result=second_modell.predict(dataFrameLatestAfter[-1:])
    return result
    
def predict(time,month,day):
    time_to_sek=lambda x: x.hour * 3600 + x.minute * 60 + x.second
    check_in_time=time_to_sek(time)
    av_time=f_modell(time)
    av_people=sec_modell(month,day)
    result=((av_time*av_people))
    return av_people,sek_to_time(check_in_time+result) #returns the av_people waitung accordinmg to the month and day and the second one return the predicted waiting time

In [198]:
result=predict(datetime.time(13,35,12),"November","Monday")
print(result)

1/1 [==============================] - 0s 14ms/step
(array([[19.953194]], dtype=float32), (array([[28.]]), array([[34.]]), array([[45.38886261]])))
